In [2]:
# WAR
# Definition: WAR measures a player's value in all facets of the game by deciphering 
# how many more wins he's worth than a replacement-level player at his position

In [3]:
# using the stat from one season to predict the player's WAR of the next season

In [4]:
pip install pybaseball

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os # python library to interact with parts of the system
import pandas as pd # data manipulation and analysis library
import numpy as np # create and work with arrays of data
from pybaseball import batting_stats # python package that lets us download baseball stats

In [6]:
# working with baseball stats from 2002 to 2022
START = 2002
END = 2022

In [7]:
# downloading the data
batting = batting_stats(START, END, qual=200) # qual is at minimum how many plate appearance a batter must have

In [8]:
# dump to a csv file
batting.to_csv("batting.csv")

In [9]:
# remove players with only one season of data
# we are making predictions about a player's WAR for their next season
#IDfg = ID in fangraphs
#groupby splits dataframe up into groups based on the value in the column
#if group_keys not set to false, pandas will change the index to include values from the field you are grouping on
batting = batting.groupby("IDfg", group_keys=False).filter(lambda x: x.shape[0] > 1) # remove groups with only 1 season of data

In [10]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,1109,2002,Barry Bonds,SFG,37,143,403,612,149,70,...,NaN,NaN,NaN,0,0.127,0.191,NaN,NaN,NaN,12.7
1,1109,2004,Barry Bonds,SFG,39,147,373,617,135,60,...,NaN,NaN,NaN,0,0.124,0.164,NaN,NaN,NaN,11.9
8,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,...,118.4,246.0,0.609,404,0.169,0.287,NaN,NaN,NaN,11.6
2,1109,2003,Barry Bonds,SFG,38,130,390,550,133,65,...,NaN,NaN,NaN,0,0.135,0.223,NaN,NaN,NaN,10.2
15,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,...,110.6,217.0,0.500,434,0.220,0.270,NaN,NaN,NaN,10.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7042,9272,2018,Chris Davis,BAL,32,128,470,522,79,51,...,111.8,113.0,0.401,282,0.174,0.316,NaN,NaN,NaN,-2.6
6535,45,2012,Rod Barajas,PIT,36,104,321,361,66,44,...,NaN,0.0,NaN,0,0.147,0.258,NaN,NaN,NaN,-2.6
6673,319,2011,Adam Dunn,CHW,31,122,415,496,66,39,...,NaN,0.0,NaN,0,0.169,0.295,NaN,NaN,NaN,-2.9
6988,620,2002,Neifi Perez,KCR,29,145,554,585,131,104,...,NaN,NaN,NaN,0,0.130,0.187,NaN,NaN,NaN,-2.9


In [11]:
# creating a ML target -> setting up a target we are trying to predict
# split data up by player again, for each player backfill the WAR value from the next season as the target
def next_season(player):
    player = player.sort_values("Season")
    player["Next_WAR"] = player["WAR"].shift(-1) # WAR shifted back one row
    return player

batting = batting.groupby("IDfg", group_keys=False).apply(next_season)

C:\Users\pingc\AppData\Local\Temp\ipykernel_892\2082164965.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  batting = batting.groupby("IDfg", group_keys=False).apply(next_season)


In [12]:
# Next_WAR is the WAR for next season, pulled value back to a year ago
batting[["Name", "Season", "WAR", "Next_WAR"]]

,Name,Season,WAR,Next_WAR
5562,Alfredo Amezaga,2006,1.1,2.0
5006,Alfredo Amezaga,2007,2.0,1.2
5252,Alfredo Amezaga,2008,1.2,NaN
1169,Garret Anderson,2002,3.7,5.1
864,Garret Anderson,2003,5.1,0.8
...,...,...,...,...
6002,Owen Miller,2022,0.8,NaN
4881,Andrew Vaughn,2021,-0.2,-0.5
3377,Andrew Vaughn,2022,-0.5,NaN
6620,Ha-seong Kim,2021,0.4,3.7


In [13]:
# Cleaning the Data
# get rid of columns with NULL values
null_count = batting.isnull().sum() # count up how many missing values are in each column

In [14]:
null_count

IDfg           0
Season         0
Name           0
Team           0
Age            0
            ... 
xBA         6754
xSLG        6754
xwOBA       6754
L-WAR          0
Next_WAR    1179
Length: 321, dtype: int64

In [15]:
# all columns that don't have missing values
complete_cols = list(batting.columns[null_count == 0]) # by default it is a pandas index

In [16]:
complete_cols

['IDfg',
 'Season',
 'Name',
 'Team',
 'Age',
 'G',
 'AB',
 'PA',
 'H',
 '1B',
 '2B',
 '3B',
 'HR',
 'R',
 'RBI',
 'BB',
 'IBB',
 'SO',
 'HBP',
 'SF',
 'SH',
 'GDP',
 'SB',
 'CS',
 'AVG',
 'GB',
 'FB',
 'LD',
 'IFFB',
 'Pitches',
 'Balls',
 'Strikes',
 'IFH',
 'BU',
 'BUH',
 'BB%',
 'K%',
 'BB/K',
 'OBP',
 'SLG',
 'OPS',
 'ISO',
 'BABIP',
 'GB/FB',
 'LD%',
 'GB%',
 'FB%',
 'IFFB%',
 'HR/FB',
 'IFH%',
 'BUH%',
 'wOBA',
 'wRAA',
 'wRC',
 'Bat',
 'Rep',
 'Pos',
 'RAR',
 'WAR',
 'Dol',
 'Spd',
 'wRC+',
 'WPA',
 '-WPA',
 '+WPA',
 'RE24',
 'REW',
 'pLI',
 'PH',
 'WPA/LI',
 'Clutch',
 'FB% (Pitch)',
 'FBv',
 'SL%',
 'SLv',
 'CB%',
 'CBv',
 'CH%',
 'CHv',
 'wFB',
 'wSL',
 'wCB',
 'wCH',
 'wFB/C',
 'wSL/C',
 'wCB/C',
 'wCH/C',
 'O-Swing%',
 'Z-Swing%',
 'Swing%',
 'O-Contact%',
 'Z-Contact%',
 'Contact%',
 'Zone%',
 'F-Strike%',
 'SwStr%',
 'BsR',
 'Def',
 'wSB',
 'UBR',
 'Age Rng',
 'Off',
 'Lg',
 'wGDP',
 'Pull%',
 'Cent%',
 'Oppo%',
 'Soft%',
 'Med%',
 'Hard%',
 'TTO%',
 'AVG+',
 'BB%+',
 'K

In [17]:
# remove null columns by indexing the batting dataframe
# creating a copy to avoid warnings
batting = batting[complete_cols + ["Next_WAR"]].copy()

In [18]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,Next_WAR
5562,1,2006,Alfredo Amezaga,FLA,28,132,334,378,87,72,...,107,113,143,109,63,0,0.188,0.256,1.1,2.0
5006,1,2007,Alfredo Amezaga,FLA,29,133,400,448,105,80,...,101,112,109,113,75,0,0.175,0.227,2.0,1.2
5252,1,2008,Alfredo Amezaga,FLA,30,125,311,337,82,61,...,101,101,123,111,64,0,0.178,0.244,1.2,NaN
1169,2,2002,Garret Anderson,ANA,30,158,638,678,195,107,...,91,80,65,97,129,0,0.137,0.232,3.7,5.1
864,2,2003,Garret Anderson,ANA,31,159,638,673,201,119,...,101,80,90,99,109,0,0.164,0.252,5.1,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002,24655,2022,Owen Miller,CLE,25,130,424,472,103,70,...,111,97,131,100,83,340,0.188,0.266,0.7,NaN
4881,26197,2021,Andrew Vaughn,CHW,23,127,417,469,98,61,...,104,116,84,99,110,321,0.185,0.285,-0.4,-0.5
3377,26197,2022,Andrew Vaughn,CHW,24,134,510,555,138,92,...,106,111,94,100,104,419,0.201,0.291,-0.5,NaN
6620,27506,2021,Ha-seong Kim,SDP,25,117,267,298,54,32,...,99,59,137,96,88,201,0.216,0.303,0.5,3.7


In [19]:
# checking the data types of the columns
# can only use numbers in machine learning algorithms (don't work with strings)
batting.dtypes

IDfg          int64
Season        int64
Name         object
Team         object
Age           int64
             ...   
Events        int64
CStr%       float64
CSW%        float64
L-WAR       float64
Next_WAR    float64
Length: 133, dtype: object

In [20]:
batting.dtypes[batting.dtypes == "object"] # finding all columns that have string/object data type

Name       object
Team       object
Dol        object
Age Rng    object
dtype: object

In [21]:
# fangraphs assign dollar value to the players, don't really need it
# since all the stats needed to compute this column are also in the dataframe
batting["Dol"]

5562      $5.5
5006     $11.2
5252      $7.2
1169     $14.6
864      $22.0
         ...  
6002      $6.1
4881    ($1.5)
3377    ($4.2)
6620      $3.0
4396     $29.8
Name: Dol, Length: 6754, dtype: object

In [22]:
del batting["Dol"]

In [23]:
# age range that the player had during the season, some players may have their birthday during the season
batting["Age Rng"]

5562    28 - 28
5006    29 - 29
5252    30 - 30
1169    30 - 30
864     31 - 31
         ...   
6002    25 - 25
4881    23 - 23
3377    24 - 24
6620    25 - 25
4396    26 - 26
Name: Age Rng, Length: 6754, dtype: object

In [24]:
del batting["Age Rng"]

In [25]:
# process team name, turn strings into numbers
batting["Team"].astype("category") #take team name and turn it into a categorical type in pandas

5562    FLA
5006    FLA
5252    FLA
1169    ANA
864     ANA
       ... 
6002    CLE
4881    CHW
3377    CHW
6620    SDP
4396    SDP
Name: Team, Length: 6754, dtype: category
Categories (35, object): ['- - -', 'ANA', 'ARI', 'ATL', ..., 'TBR', 'TEX', 'TOR', 'WSN']

In [26]:
# convert each category into a number
batting["Team"].astype("category").cat.codes

5562    12
5006    12
5252    12
1169     1
864      1
        ..
6002     9
4881     7
3377     7
6620    26
4396    26
Length: 6754, dtype: int8

In [27]:
# assigning to a variable called team_code
batting["team_code"] = batting["Team"].astype("category").cat.codes

In [36]:
# make a copy of batting data
# copy to another variable because any rows where the Next_WAR is missing will be dropped 
# missing rows may still be importing to predicting the future
batting_full = batting.copy()
batting = batting.dropna().copy() # drop rows where Next_WAR is None, copy avoids error, when you manipulate your df too many times

In [31]:
pip install scikit-learn

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB 653.6 kB/s eta 0:00:17
    --------------------------------------- 0.2/11.0 MB 1.5 MB/s eta 0:00:08
   - -------------------------------------- 0.3/11.0 MB 2.1 MB/s eta 0:00:06
   -- ------------------------------------- 0.6/11.0 MB 3.2 MB/s eta 0:00:04
   ---- ----------------------------------- 1.2/11.0 MB 4.9 MB/s eta 0:00:03
   ----- ---------------------------------- 1.6/11.0 MB 5.8 MB/s eta 0:00:02
   ------- -------------------------------- 1.9/11.0 MB 6.5 MB/s eta 0:00:02
   ------- -------------------------------- 1.9/11.0 MB 6.5 MB/s eta 0:00:02
   ------------- -------------------------- 3.8/11.0 MB 8.9 MB/s eta 0:00:01
   --------------- ------------------------ 4.2/11.0 MB 9.0 MB/s eta 0:00:01
   ------------------ --------------------- 5.0/11.0 MB 9.7 MB/s eta 0:00:01
   --------------------- ------------------ 5.9/11.0 MB 10.4 MB/s eta 0:00:01
   

In [37]:
# Selecting Useful Features: Don't want to fit all columns inside the machine learning algorithm
# may cause overfitting
# feature selector: can pick a subset of features that help optimize the accuracy of the model

from sklearn.linear_model import Ridge # using a ridge regression model
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

# initialize ridge regression model, alpha is lambda in python
rr = Ridge(alpha=1) # higher alpha reduces overfitting b/c it penalizes the ridge regression coefficient

# initialize time series split, don't want to use future data to predict something from the past
split = TimeSeriesSplit(n_splits=3) # split data up into 3 parts and make predictions for those parts using a time-series aware way

# initialize sequential feature selector
# pass in ridge regression model, define how many features you want selected, using time series split 
# n_jobs is using multiple threads to process faster (don't need it)
# direction=forward: start by selecting 0 features and keep evaluating all the features to find the best ones
sfs = SequentialFeatureSelector(rr, n_features_to_select=20, direction="forward", cv=split, n_jobs=4)


In [38]:
# sequential feature selector doesn't work with some columns, don't want to pass those in
# no target field, text columns and take out meta deta fields (might overfit)
removed_columns = ["Next_WAR", "Name", "Team", "IDfg", "Season"]
# create a list to be all the columns except ones removed
selected_columns = batting.columns[~batting.columns.isin(removed_columns)]

In [39]:
# for ridge regression model, scale data so mean is 0 and standard deviation is 1 in order for model to work effectively
# min-max scaling: more aggressive form of scaling to find ratios of columns later on, will put all values between 0 and 1
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
batting.loc[:, selected_columns] = scaler.fit_transform(batting[selected_columns])

In [40]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,Next_WAR,team_code
5562,1,2006,Alfredo Amezaga,FLA,0.346154,0.735043,0.312950,0.307958,0.245690,0.278302,...,0.503759,0.662921,0.652174,0.210884,0.000000,0.582979,0.524229,0.265823,2.0,0.352941
5006,1,2007,Alfredo Amezaga,FLA,0.384615,0.743590,0.431655,0.429066,0.323276,0.316038,...,0.496241,0.471910,0.710145,0.292517,0.000000,0.527660,0.396476,0.322785,1.2,0.352941
1169,2,2002,Garret Anderson,ANA,0.423077,0.957265,0.859712,0.826990,0.711207,0.443396,...,0.255639,0.224719,0.478261,0.659864,0.000000,0.365957,0.418502,0.430380,5.1,0.029412
864,2,2003,Garret Anderson,ANA,0.461538,0.965812,0.859712,0.818339,0.737069,0.500000,...,0.255639,0.365169,0.507246,0.523810,0.000000,0.480851,0.506608,0.518987,0.8,0.029412
2569,2,2004,Garret Anderson,ANA,0.500000,0.564103,0.507194,0.475779,0.443966,0.400943,...,0.218045,0.297753,0.608696,0.448980,0.000000,0.531915,0.585903,0.246835,-0.2,0.029412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914,23667,2021,Wander Franco,TBR,0.038462,0.205128,0.217626,0.186851,0.219828,0.179245,...,0.390977,0.421348,0.608696,0.394558,0.409015,0.391489,0.352423,0.348101,2.3,0.911765
5875,24618,2021,Ryan Jeffers,MIN,0.192308,0.333333,0.192446,0.160900,0.099138,0.070755,...,0.315789,0.376404,0.347826,0.619048,0.265442,0.514894,0.788546,0.240506,0.8,0.558824
7032,24655,2021,Owen Miller,CLE,0.192308,0.119658,0.055755,0.003460,0.038793,0.066038,...,0.593985,0.331461,0.681159,0.394558,0.230384,0.548936,0.700441,0.139241,0.8,0.264706
4881,26197,2021,Andrew Vaughn,CHW,0.153846,0.692308,0.462230,0.465398,0.293103,0.226415,...,0.526316,0.331461,0.507246,0.530612,0.535893,0.570213,0.651982,0.170886,-0.5,0.205882


In [43]:
# take a look at the changes in a more summarized way
batting.describe()

,IDfg,Season,Age,G,AB,PA,H,1B,2B,3B,...,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,Next_WAR,team_code
count,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,...,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000
mean,5366.785830,2011.163229,0.360600,0.652755,0.478666,0.480943,0.365973,0.290481,0.399279,0.103459,...,0.403164,0.410923,0.511026,0.478646,0.172991,0.498932,0.545898,0.322045,1.793291,0.474128
std,5133.255295,5.612014,0.147476,0.255929,0.242481,0.262290,0.182585,0.138786,0.171732,0.105891,...,0.131213,0.121082,0.130359,0.133992,0.273858,0.137180,0.120701,0.122149,1.981340,0.305105
min,1.000000,2002.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.100000,0.000000
25%,1131.500000,2006.000000,0.269231,0.478632,0.275180,0.257785,0.211207,0.179245,0.258621,0.043478,...,0.315789,0.331461,0.420290,0.387755,0.000000,0.408511,0.466960,0.234177,0.400000,0.205882
50%,3531.000000,2011.000000,0.346154,0.709402,0.505396,0.508651,0.370690,0.283019,0.379310,0.086957,...,0.398496,0.404494,0.507246,0.489796,0.000000,0.493617,0.546256,0.303797,1.500000,0.470588
75%,9015.000000,2016.000000,0.461538,0.871795,0.688849,0.710208,0.508621,0.391509,0.517241,0.130435,...,0.488722,0.483146,0.594203,0.564626,0.346411,0.591489,0.625551,0.392405,2.900000,0.735294
max,27506.000000,2021.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.900000,1.000000


In [44]:
# apply sequential feature selector
# fit method picks the 20 predictors that give us the best accuracy with the ridge regression model
sfs.fit(batting[selected_columns], batting["Next_WAR"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=20,
                          n_jobs=4)

In [46]:
# extract list of predictors from the sequential feature selector
# select the True columns
sfs.get_support()

array([ True, False, False, False, False, False, False, False, False,
       False, False, False,  True,  True, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False,  True,  True, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True, False,  True, False, False, False, False, False,
        True, False, False, False, False, False, False,  True, False,
        True, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
        True,  True, False, False, False, False, False, False,  True,
       False, False,  True, False,  True, False, False, False, False,
       False])

In [48]:
# index selected columns list
selected_columns[sfs.get_support()]

Index(['Age', 'IBB', 'SO', 'SB', 'IFH', 'BU', 'BABIP', 'WAR', 'Spd', 'WPA',
       'PH', 'CB%', 'CH%', 'wCH', 'Oppo%', 'OBP+', 'SLG+', 'Pull%+', 'Soft%+',
       'Hard%+'],
      dtype='object')

In [49]:
# convert to a flat python list
predictors = list(selected_columns[sfs.get_support()])

In [50]:
sorted(batting["Season"].unique())

[np.int64(2002),
 np.int64(2003),
 np.int64(2004),
 np.int64(2005),
 np.int64(2006),
 np.int64(2007),
 np.int64(2008),
 np.int64(2009),
 np.int64(2010),
 np.int64(2011),
 np.int64(2012),
 np.int64(2013),
 np.int64(2014),
 np.int64(2015),
 np.int64(2016),
 np.int64(2017),
 np.int64(2018),
 np.int64(2019),
 np.int64(2020),
 np.int64(2021)]

In [53]:
# Making Predictions with ML
# backtest generates the predictions
# cross validation splits data up into several groups, ex: for group 1, use 2 and 3 to train the algorithm, and predict on group 1
# cross validation doesn't work with timeseries data, we don't want to use data from future seasons to predict data from past seasons
# only use past data to predict future data
def backtest(data, model, predictors, start=5, step=1):
    all_predictions = [] # each element is a prediction for each individual season
    years = sorted(data["Season"].unique()) # find all unique seasons
    for i in range(start, len(years), step): # each time through the loop use historical data to predict a single season
        current_year = years[i]
        train = data[data["Season"] < current_year] # training set is everything where the season is less than the current year
        test = data[data["Season"] == current_year] # test set is anything where the season is the current year

        model.fit(train[predictors], train["Next_WAR"]) # fit model using the training predictors as well as the target trying to predict
        preds = model.predict(test[predictors]) # use predict method to generate predictions on the test set
        preds = pd.Series(preds, index=test.index) # turn numpy array into a pandas series, easier to work with
        # pandas series is like a single column in a dataframe
        # combine the predictions with actual values
        combined = pd.concat([test["Next_WAR"], preds], axis=1) # axis=1 returns two separate columns
        combined.columns = ["actual", "prediction"]

        all_predictions.append(combined)
        # at the end, all predictions will be a list of dataframes, and each dataframe will be the prediction for a season
    return pd.concat(all_predictions) # combining all the dataframes vertically

In [54]:
predictions = backtest(batting, rr, predictors)

In [55]:
predictions

,actual,prediction
5006,1.2,1.405835
1925,1.4,0.716105
3102,-0.1,0.457908
5797,0.6,0.979155
1109,4.8,2.214873
...,...,...
1914,2.3,2.753679
5875,0.8,2.083942
7032,0.8,1.583851
4881,-0.5,1.820774


In [56]:
# use summary statistic to create an error metric
from sklearn.metrics import mean_squared_error # returns a single number to show how high the error is in the model

# subtract the prediction from the actual value and squared the difference, find average squared difference across all rows
mean_squared_error(predictions["actual"], predictions["prediction"])

np.float64(2.738079335969731)

In [57]:
batting["Next_WAR"].describe()

count    5575.000000
mean        1.793291
std         1.981340
min        -3.100000
25%         0.400000
50%         1.500000
75%         2.900000
max        11.900000
Name: Next_WAR, dtype: float64

In [58]:
# we want square root of mean_squared_error to be lower than the std, indicates the model is doing better than random guessing
2.738079335969731 ** .5

1.65471427623313

In [65]:
# Garret Anderson example
# select player's seasons
ga = batting[batting["IDfg"] == 2].copy()

In [66]:
ga

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,Next_WAR,team_code
1169,2,2002,Garret Anderson,ANA,0.423077,0.957265,0.859712,0.826990,0.711207,0.443396,...,0.255639,0.224719,0.478261,0.659864,0.0,0.365957,0.418502,0.430380,5.1,0.029412
864,2,2003,Garret Anderson,ANA,0.461538,0.965812,0.859712,0.818339,0.737069,0.500000,...,0.255639,0.365169,0.507246,0.523810,0.0,0.480851,0.506608,0.518987,0.8,0.029412
2569,2,2004,Garret Anderson,ANA,0.500000,0.564103,0.507194,0.475779,0.443966,0.400943,...,0.218045,0.297753,0.608696,0.448980,0.0,0.531915,0.585903,0.246835,-0.2,0.029412
4187,2,2005,Garret Anderson,LAA,0.538462,0.820513,0.746403,0.697232,0.573276,0.462264,...,0.278195,0.421348,0.478261,0.503401,0.0,0.421277,0.533040,0.183544,0.1,0.441176
3964,2,2006,Garret Anderson,LAA,0.576923,0.811966,0.688849,0.671280,0.525862,0.433962,...,0.300752,0.353933,0.434783,0.591837,0.0,0.442553,0.511013,0.202532,1.4,0.441176
1925,2,2007,Garret Anderson,LAA,0.615385,0.529915,0.462230,0.432526,0.405172,0.297170,...,0.285714,0.443820,0.420290,0.523810,0.0,0.442553,0.480176,0.284810,1.4,0.441176
3346,2,2008,Garret Anderson,LAA,0.653846,0.846154,0.714029,0.679931,0.573276,0.495283,...,0.285714,0.387640,0.565217,0.442177,0.0,0.527660,0.533040,0.284810,-1.1,0.441176


In [67]:
# create a player season column
ga["player_season"] = range(0, ga.shape[0])

In [68]:
ga

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,Next_WAR,team_code,player_season
1169,2,2002,Garret Anderson,ANA,0.423077,0.957265,0.859712,0.826990,0.711207,0.443396,...,0.224719,0.478261,0.659864,0.0,0.365957,0.418502,0.430380,5.1,0.029412,0
864,2,2003,Garret Anderson,ANA,0.461538,0.965812,0.859712,0.818339,0.737069,0.500000,...,0.365169,0.507246,0.523810,0.0,0.480851,0.506608,0.518987,0.8,0.029412,1
2569,2,2004,Garret Anderson,ANA,0.500000,0.564103,0.507194,0.475779,0.443966,0.400943,...,0.297753,0.608696,0.448980,0.0,0.531915,0.585903,0.246835,-0.2,0.029412,2
4187,2,2005,Garret Anderson,LAA,0.538462,0.820513,0.746403,0.697232,0.573276,0.462264,...,0.421348,0.478261,0.503401,0.0,0.421277,0.533040,0.183544,0.1,0.441176,3
3964,2,2006,Garret Anderson,LAA,0.576923,0.811966,0.688849,0.671280,0.525862,0.433962,...,0.353933,0.434783,0.591837,0.0,0.442553,0.511013,0.202532,1.4,0.441176,4
1925,2,2007,Garret Anderson,LAA,0.615385,0.529915,0.462230,0.432526,0.405172,0.297170,...,0.443820,0.420290,0.523810,0.0,0.442553,0.480176,0.284810,1.4,0.441176,5
3346,2,2008,Garret Anderson,LAA,0.653846,0.846154,0.714029,0.679931,0.573276,0.495283,...,0.387640,0.565217,0.442177,0.0,0.527660,0.533040,0.284810,-1.1,0.441176,6


In [69]:
ga[["player_season", "WAR"]]

,player_season,WAR
1169,0,0.430380
864,1,0.518987
2569,2,0.246835
4187,3,0.183544
3964,4,0.202532
1925,5,0.284810
3346,6,0.284810


In [74]:
# expanding creates different groups of the dataframe: first group is first row, second group is the first two rows...
# for each group in the expanding, find correlation between player season and WAR
# ex: third group that expanding creates, look at the correlation between 0, 1, 2 and their WAR values
ga[["player_season", "WAR"]].expanding().corr()

player_season       WAR
1169 player_season            NaN       NaN
     WAR                      NaN       NaN
864  player_season       1.000000  1.000000
     WAR                 1.000000  1.000000
2569 player_season       1.000000 -0.661143
     WAR                -0.661143  1.000000
4187 player_season       1.000000 -0.836562
     WAR                -0.836562  1.000000
3964 player_season       1.000000 -0.836312
     WAR                -0.836312  1.000000
1925 player_season       1.000000 -0.692192
     WAR                -0.692192  1.000000
3346 player_season       1.000000 -0.595013
     WAR                -0.595013  1.000000

In [76]:
# correlation returns 4 numbers for each row, each row is represented by the index on the left, two rows + two columns
# want just a single number using loc indexer
# multi-index: first level are the numbers on the left, second level is the player_season and WAR
# slice(None) = select all values from level 1 of the index, level 2 select player_season, and only select the WAR column
ga[["player_season", "WAR"]].expanding().corr().loc[(slice(None), "player_season"), "WAR"]

1169  player_season         NaN
864   player_season    1.000000
2569  player_season   -0.661143
4187  player_season   -0.836562
3964  player_season   -0.836312
1925  player_season   -0.692192
3346  player_season   -0.595013
Name: WAR, dtype: float64

In [79]:
# turn into list
list(ga[["player_season", "WAR"]].expanding().corr().loc[(slice(None), "player_season"), "WAR"])
# for every season that ga player, this gives the correlation between his season number and his WAR for all previous seasons

[nan,
 1.0,
 -0.6611430912519526,
 -0.8365619976685158,
 -0.8363121929961227,
 -0.6921918007562199,
 -0.5950132649769159]

In [81]:
# Improving Accuracy
# improve prediction by giving algorithm some information on how the player did in previous seasons
def player_history(df):
    df = df.sort_values("Season")
    # create predictors
    df["player_season"] = range(0, df.shape[0]) # which season it is for the player
    # wins above replacement correlation
    df["war_corr"] = list(df[["player_season", "WAR"]].expanding().corr().loc[(slice(None), "player_season"), "WAR"])
    df["war_corr"].fillna(1, inplace=True)
    # difference(ratio) between current WAR and the previous season's WAR
    # shift(-1): brings next season's value back to the current season
    # shift(1): brings the previous season's value up to the current season
    df["war_diff"] = df["WAR"] / df["WAR"].shift(1) #takes the current WAR and divide it by the WAR of the previous season
    df["war_diff"].fillna(1, inplace=True) # 1 assumes WAR has been constant from last season
    # division by 0 causes inf, find any value in the war_diff column that are infinite and replace them with a 1
    df["war_diff"][df["war_diff"] ==np.inf] = 1

    return df
# grouping by player, then for each player calling the player_history function and passing in data for that player
batting = batting.groupby("IDfg", group_keys=False).apply(player_history)

C:\Users\pingc\AppData\Local\Temp\ipykernel_892\376846062.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["war_corr"].fillna(1, inplace=True)
C:\Users\pingc\AppData\Local\Temp\ipykernel_892\376846062.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doin

In [82]:
# find averages across the whole season and compare those averages to the players
# did the player perform better than the average player or worse
# helps correct if players played less games (lock out)
def group_averages(df):
    return df["WAR"] / df["WAR"].mean()

In [84]:
# group by season, 1 group for each season, and apply group_averages function
# find the average between how each player did, and how the average player did for each season
batting["war_season"] = batting.groupby("Season", group_keys=False).apply(group_averages)

C:\Users\pingc\AppData\Local\Temp\ipykernel_892\3345959431.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  batting["war_season"] = batting.groupby("Season", group_keys=False).apply(group_averages)


In [85]:
new_predictors = predictors + ["player_season", "war_corr", "war_season", "war_diff"]

In [86]:
predictions = backtest(batting, rr, new_predictors)

In [87]:
mean_squared_error(predictions["actual"], predictions["prediction"])

np.float64(2.6807349837451784)

In [88]:
# Diagnosing Issues With The Model
# look at how much each predictor is impacting the model by looking at the coefficients of the ridge regression model
rr.coef_

array([-2.71350921e+00,  2.05514516e+00, -8.98257728e-01,  9.66494234e-01,
        6.34806760e-01, -1.12984275e+00, -1.94505118e+00, -1.84860791e+00,
        7.68120976e-01, -5.54977543e-01, -7.43875249e-01, -2.73378585e-01,
       -2.88841765e-01, -2.85613438e-01,  6.92382017e-01,  8.37156389e-01,
       -1.38114396e+00, -2.20954194e-01, -1.32036143e+00,  2.44191614e+00,
        2.25442539e-04, -1.37501195e-01,  3.19125472e+00, -2.84179470e-01])

In [90]:
# combine with the names of the predictors to see what is happening
# small coefficient: model not really taking into account
pd.Series(rr.coef_, index=new_predictors).sort_values()

Age             -2.713509
BABIP           -1.945051
WAR             -1.848608
SLG+            -1.381144
Soft%+          -1.320361
BU              -1.129843
SO              -0.898258
PH              -0.743875
WPA             -0.554978
CH%             -0.288842
wCH             -0.285613
war_diff        -0.284179
CB%             -0.273379
Pull%+          -0.220954
war_corr        -0.137501
player_season    0.000225
IFH              0.634807
Oppo%            0.692382
Spd              0.768121
OBP+             0.837156
SB               0.966494
IBB              2.055145
Hard%+           2.441916
war_season       3.191255
dtype: float64

In [91]:
# look at the difference between actual values and the predictions
diff = predictions["actual"] - predictions["prediction"]

In [92]:
diff

5006   -0.222652
1925    0.909449
3102   -0.346150
5797   -0.386085
1109    2.842180
          ...   
1914   -0.349174
5875   -1.163408
7032   -0.566911
4881   -2.064754
6620    2.764180
Length: 4127, dtype: float64

In [93]:
# merge dataframes to see the difference along with the other stats
# using index to merge the prediction with the batting dataframe
merged = predictions.merge(batting, left_index=True, right_index=True)

In [94]:
merged["diff"] = (predictions["actual"] - predictions["prediction"]).abs()

In [95]:
merged

,actual,prediction,IDfg,Season,Name,Team,Age,G,AB,PA,...,CStr%,CSW%,L-WAR,Next_WAR,team_code,player_season,war_corr,war_diff,war_season,diff
5006,1.2,1.422652,1,2007,Alfredo Amezaga,FLA,0.384615,0.743590,0.431655,0.429066,...,0.527660,0.396476,0.322785,1.2,0.352941,1,1.000000,1.214286,0.998259,0.222652
1925,1.4,0.490551,2,2007,Garret Anderson,LAA,0.615385,0.529915,0.462230,0.432526,...,0.442553,0.480176,0.284810,1.4,0.441176,5,-0.692192,1.406250,0.880816,0.909449
3102,-0.1,0.246150,10,2007,David Eckstein,STL,0.500000,0.606838,0.492806,0.491349,...,0.676596,0.436123,0.240506,-0.1,0.852941,5,-0.694330,0.826087,0.743801,0.346150
5797,0.6,0.986085,11,2007,Darin Erstad,CHW,0.538462,0.350427,0.269784,0.254325,...,0.765957,0.691630,0.240506,0.6,0.205882,4,-0.828562,0.791667,0.743801,0.386085
1109,4.8,1.957820,15,2007,Troy Glaus,TOR,0.423077,0.589744,0.404676,0.442907,...,0.634043,0.704846,0.367089,4.8,0.970588,5,0.231396,0.892308,1.135274,2.842180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914,2.3,2.649174,23667,2021,Wander Franco,TBR,0.038462,0.205128,0.217626,0.186851,...,0.391489,0.352423,0.348101,2.3,0.911765,0,1.000000,1.000000,1.061285,0.349174
5875,0.8,1.963408,24618,2021,Ryan Jeffers,MIN,0.192308,0.333333,0.192446,0.160900,...,0.514894,0.788546,0.240506,0.8,0.558824,0,1.000000,1.000000,0.829732,1.163408
7032,0.8,1.366911,24655,2021,Owen Miller,CLE,0.192308,0.119658,0.055755,0.003460,...,0.548936,0.700441,0.139241,0.8,0.264706,0,1.000000,1.000000,0.463106,0.566911
4881,-0.5,1.564754,26197,2021,Andrew Vaughn,CHW,0.153846,0.692308,0.462230,0.465398,...,0.570213,0.651982,0.170886,-0.5,0.205882,0,1.000000,1.000000,0.559587,2.064754


In [98]:
# filtering of columns
merged[["IDfg", "Season", "Name", "WAR", "Next_WAR", "diff"]].sort_values(["diff"])

,IDfg,Season,Name,WAR,Next_WAR,diff
6023,4403,2013,Erik Kratz,0.246835,1.1,0.001375
1190,15172,2019,Tim Anderson,0.481013,2.3,0.002997
3266,1286,2008,Michael Young,0.348101,2.6,0.003986
2082,5887,2013,John Jaso,0.234177,0.6,0.004329
2159,1702,2008,Reed Johnson,0.284810,0.3,0.004360
...,...,...,...,...,...,...
3823,1875,2009,Josh Hamilton,0.278481,8.4,6.457327
871,9166,2010,Buster Posey,0.443038,9.8,6.526769
3245,5631,2010,Matt Kemp,0.196203,8.3,6.526948
451,15640,2021,Aaron Judge,0.544304,11.2,7.417640
